In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 2.5 MB/s 
     |████████████████████████████████| 3.1 MB 9.3 MB/s 
     |████████████████████████████████| 61 kB 302 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=8da9337254c7f20d556ec5a968ab0c7f22bf9d25eb6936398add3e816958f69f
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: transformers
    Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from collections import Counter

import gc
import cloudpickle

import os, sys
import random

import warnings

import xgboost as xgb


import base64
import io

import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
from PIL import Image

from tqdm import tqdm

print('Using \033[34mXGBoost', xgb.__version__, '\033[0m')

Using XGBoost 1.5.0 


In [3]:
model = SentenceTransformer('clip-ViT-B-32')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/605M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/961k [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [4]:
def transform2image_embeddings(df):
    image2text_embeddings = []
    widths = []
    heights = []

    for k in tqdm(range(df.shape[0])):
        data = df.loc[k, 1]
        
        try:
            image = base64.b64decode(data)
            img = Image.open(io.BytesIO(image)).convert('RGB')
        
            width, height = img.size
            widths.append(width)
            heights.append(height)

            image2text_embeddings.append(model.encode(img, show_progress_bar=False))
        except:
            print(k)
            widths.append(np.nan)
            heights.append(np.nan)
            image2text_embeddings.append(np.full(512, np.nan))
            
        
    image2text_embeddings = np.vstack(image2text_embeddings)
    
    embeddings_df = pd.DataFrame(image2text_embeddings)
    embeddings_df.insert(loc=0, column='image_id', value=df['image_id'])
    
    size_df = pd.DataFrame({'image_id': df['image_id'], 'width': widths, 'height': heights})
    
    return embeddings_df, size_df

In [5]:
import requests


def prepare_train_embeddings(images_path, indices):
    images = pd.read_csv(images_path)
    images['image_id'] = images.index
    
    container = []
    sizes = []
    
    found = 0
    progress_bar = indices

    for index in progress_bar:
        print('\033[34mHandling part {}...\033[0m'.format(index))
        
#         url = 'https://analytics.wikimedia.org/published/datasets/one-off/caption_competition/training/image_pixels/part-00{0:03d}-04b253b8-db8c-4d14-a23f-3433a86841b4-c000.csv.gz'.format(index)
#         r = requests.get(url, allow_redirects=True)

#         with open('pixels.csv', 'wb') as file:
#             file.write(r.content)

        df = pd.read_csv('../input/wikipedia-image-caption/image_data_test/image_pixels/test_image_pixels_part-00{0:03d}.csv'.format(index), sep='\t', header=None)
        df.columns = ['image_url', 1, 2]
        df = pd.merge(images, df, on='image_url')
        df, size_df = transform2image_embeddings(df)
        
        gc.collect()
        container.append(df)
        sizes.append(size_df)
        
        found += df.shape[0]
#         progress_bar.set_description('Found: {0:.2f}'.format(found / images.shape[0]))

        print('found: {0:.2f}'.format(found / images.shape[0]))
        print()
        
        if found == images.shape[0]:
            break
    
    container = pd.concat(container)
    sizes = pd.concat(sizes)
    
    temp = images['image_id']
    
    container = pd.merge(temp, container, on='image_id', how='left').sort_values(by='image_id')
    container.fillna(np.float32(1.), inplace=True)
    
    sizes = pd.merge(temp, sizes, on='image_id', how='left').sort_values(by='image_id')

    return container, sizes

In [6]:
%%time

embeddings, sizes = prepare_train_embeddings('../input/test-final-image-dataset/final_images.csv', range(5))

Handling part 0...


 85%|████████▍ | 7604/8952 [22:15<03:47,  5.92it/s]/opt/conda/lib/python3.7/site-packages/PIL/Image.py:963: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████| 8952/8952 [26:15<00:00,  5.68it/s]


found: 0.20

Handling part 1...


100%|██████████| 8947/8947 [26:43<00:00,  5.58it/s]


found: 0.40

Handling part 2...


...

found: 0.60

Handling part 3...


100%|██████████| 8959/8959 [27:05<00:00,  5.51it/s]


found: 0.80

Handling part 4...


100%|██████████| 8958/8958 [26:54<00:00,  5.55it/s]


found: 1.00

CPU times: user 3h 58min 14s, sys: 59.7 s, total: 3h 59min 14s
Wall time: 2h 14min 32s


In [7]:
embeddings

,image_id,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,0,-0.186673,-0.047442,0.335585,-0.364513,0.252310,-0.242793,-0.041301,0.170344,0.598797,...,-0.230711,-0.101689,-0.144105,-0.015875,-0.138280,-0.189405,-0.115695,0.250270,-0.726988,-0.305002
1,1,-0.368234,-0.230337,-0.245023,-0.134927,-0.404701,-0.600606,-0.352422,-0.296430,-0.732322,...,0.102706,-0.124401,0.533496,0.165941,-0.113144,0.023298,-0.370428,0.469564,0.344825,-0.088460
2,2,-0.081683,-0.203162,-0.062810,0.292377,0.064627,0.039117,0.012121,-0.119641,0.918837,...,-0.042029,0.322317,0.464774,0.600202,-0.137407,-0.069717,-0.173886,0.027901,0.031199,0.207190
3,3,-0.402780,0.094362,-0.264185,-0.347794,0.007422,0.188255,0.085374,0.018406,0.066538,...,-0.332526,0.286512,0.626571,0.302822,0.078901,0.041055,-0.159816,0.363164,-0.054876,-0.177849
4,4,-0.370306,0.481966,-0.233309,0.233420,-0.160380,-0.151146,-0.260257,-0.473583,-0.313270,...,0.149451,0.272180,-0.660737,-0.044660,-0.065890,0.134769,-0.146292,0.667066,0.437016,-0.129964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44757,44757,0.329485,0.113349,-0.308375,0.077601,-0.138407,0.302141,0.130387,0.262125,-0.378183,...,-0.035942,0.265228,0.737693,0.311814,0.091642,-0.393103,-0.019851,-0.065168,0.092518,0.037875
44758,44758,-0.269238,0.192454,-0.075605,0.013086,-0.337474,-0.234468,-0.596162,-0.103514,-0.591176,...,-0.340384,-0.178788,-0.002503,0.216491,-0.122229,-0.212360,0.050197,0.641916,0.262713,0.234503
44759,44759,-0.021921,0.334409,-0.179543,-0.073545,-0.274271,-0.028671,0.264936,0.214976,-0.345124,...,-0.131702,0.238574,0.569366,0.180495,-0.082951,-0.007586,0.088620,0.503842,0.092770,0.027785
44760,44760,-0.440289,0.023626,-0.174890,0.088513,0.427566,0.175322,-0.794170,-0.244655,0.010458,...,0.143178,-0.189750,0.308643,-0.069620,-0.449782,0.012805,-0.256596,0.248926,-0.080500,-0.371198


In [8]:
sizes['pixel_size_ratio'] = sizes['width'] / (sizes['height'] + 1e-3)
sizes['pixel_area'] = np.log1p(sizes['width'] * sizes['height'])

sizes.rename(columns={'width': 'pixel_width', 'height': 'pixel_height'}, inplace=True)

sizes

,image_id,pixel_width,pixel_height,pixel_size_ratio,pixel_area
0,0,300,511,0.587083,11.940159
1,1,300,491,0.610997,11.900233
2,2,300,225,1.333327,11.119898
3,3,300,225,1.333327,11.119898
4,4,300,225,1.333327,11.119898
...,...,...,...,...,...
44757,44757,300,348,0.862066,11.555995
44758,44758,300,193,1.554396,10.966490
44759,44759,300,218,1.376140,11.088293
44760,44760,300,414,0.724636,11.729656


In [9]:
sizes[sizes['pixel_width'].notna()]

,image_id,pixel_width,pixel_height,pixel_size_ratio,pixel_area
0,0,300,511,0.587083,11.940159
1,1,300,491,0.610997,11.900233
2,2,300,225,1.333327,11.119898
3,3,300,225,1.333327,11.119898
4,4,300,225,1.333327,11.119898
...,...,...,...,...,...
44757,44757,300,348,0.862066,11.555995
44758,44758,300,193,1.554396,10.966490
44759,44759,300,218,1.376140,11.088293
44760,44760,300,414,0.724636,11.729656


In [10]:
with open('image_embeddings.pickle', 'wb') as file:
    cloudpickle.dump(embeddings[range(512)].values, file)
    
sizes.to_csv('sizes.csv', index=False)